In [ ]:
%%capture
!pip install llama-index==0.10.25 llama-index-embeddings-cohere llama-index-llms-cohere qdrant-client llama-index-vector-stores-qdrant 

In [ ]:
import os
from getpass import getpass

In [ ]:
CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")

# 🕵🏻 Agents

Automated engines that process user queries, break down complex questions, select tools, set parameters, and plan tasks.

- 🧠 **Key Capabilities**:
  - Decomposing complex queries into simpler questions.
  - Selecting and parameterizing external Tools.
  - Task planning and execution.
  - Storing task history in a memory module.
  - Automate search across unstructured, semi-structured, and structured data.
  - Call external service APIs, process responses, and store information for future use.

- 🛠️ **Core Components for Building a Data Agent**:
  - A reasoning loop to make decisions based on the input.
  - Tool abstractions for interacting with APIs.
  - Initialization with a set of APIs (Tools) for data interaction and modification.

###  A Simple Intro with Calculator Tools

- Introduction to how ReAct agent operates using basic calculator tools, without complex pipelines or API integrations.

- Step-by-step reasoning process using different tools to achieve objectives.


In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.cohere import Cohere
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

# Define Function Tools

- 🔨 **Setting Up Function Tools**: Creation of simple multiply and add functions.

- 📝 **FunctionTool Usage**: Illustrates how arbitrary functions can be integrated with `FunctionTool`, using docstring and parameter signature processing.

In [ ]:
def multiply(a: int, b: int) -> int:
    """
    Multiplies two integers in an alternative universe's mathematical rules. 
    Specifically, it multiplies the second integer by 1.25 and then multiplies the result with the first integer. 
    Returns the final multiplication result as an integer.
    
    Parameters:
    a (int): The first integer to multiply.
    b (int): The second integer, which is first multiplied by 1.25 before the overall multiplication.

    Returns:
    int: The result of the alternative universe multiplication.
    """
    return a * (b * 1.25)

def add(a: int, b: int) -> int:
    """
    Adds two integers in an alternate universe's mathematical rules.
    Specifically, it subtracts 0.42 from the first integer, then adds the second integer to the result.
    This operation reflects the unique arithmetic properties of this universe.
    
    Parameters:
    a (int): The first integer, from which 0.42 is subtracted before addition.
    b (int): The second integer, added to the adjusted first integer.

    Returns:
    int: The result of the alternate universe addition, rounded to the nearest integer.
    """
    return (a - 0.42) + b

multiply_tool = FunctionTool.from_defaults(
    fn=multiply, 
    name="multiply", 
    )

add_tool = FunctionTool.from_defaults(
    fn=add,
    name="add",
    )

In [ ]:
multiply_tool.metadata.__dict__

In [ ]:
llm = Cohere(model="command-r")

agent = ReActAgent.from_tools(
    [multiply_tool, add_tool], 
    llm=llm, 
    verbose=True
    )

In [ ]:
agent_prompts = agent.get_prompts()

In [ ]:
print(agent_prompts['agent_worker:system_prompt'].template)

In [ ]:
add(multiply(3,4), 5)

In [ ]:
response = agent.chat("""You live in an alternate universe. Math works according to the tools provided. Use the provided tools to multiply 3 by 4 and add 5 to the result""")